In [1]:
# NYC Data Science Academy
# Web Scrapping Project
# meetup.com analysis and number of participants prediction
# Chia-An Chen
# 08/14/2016

In [1]:
# import modules
from bs4 import BeautifulSoup
import requests
import re
import datetime
import time
import numpy as np
import pandas as pd
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

In [2]:
def create_url_dict(radius):
    '''create url dictionary for 10 cities in the US'''
    ny_url = "http://www.meetup.com/find/events/?allMeetups=true&radius="+str(radius)+"&userFreeform=Manhattan%2C+New+York%2C+USA&mcId=c10001&mcName=New+York%2C+NY&eventFilter=all"
    sf_url = "https://www.meetup.com/find/events/?allMeetups=true&radius="+str(radius)+"&userFreeform=sanfrancisco&gcResults=San+Francisco%2C+CA%2C+USA%3AUS%3ACalifornia%3ASan+Francisco+County%3ASan+Francisco%3Anull%3Anull%3A37.7749295%3A-122.41941550000001&change=yes"
    chicago_url = "https://www.meetup.com/find/events/?allMeetups=true&radius="+str(radius)+"&userFreeform=chicago&mcId=c60601&gcResults=Chicago%2C+IL%2C+USA%3AUS%3AIllinois%3ACook+County%3AChicago%3Anull%3Anull%3A41.8781136%3A-87.62979819999998&change=yes"
    dc_url = "https://www.meetup.com/find/events/?allMeetups=true&radius="+str(radius)+"&userFreeform=washington&gcResults=Washington%2C+DC%2C+USA%3AUS%3ADistrict+of+Columbia%3ADistrict+of+Columbia%3AWashington%3Anull%3Anull%3A38.9071923%3A-77.03687070000001%7CWashington%2C+UT%2C+USA%3AUS%3AUtah%3AWashington+County%3AWashington%3Anull%3Anull%3A37.1305373%3A-113.50828669999999%7CWashington%2C+IA+52353%2C+USA%3AUS%3AIowa%3AWashington+County%3AWashington%3Anull%3A52353%3A41.30140770000001%3A-91.69164169999999%7CWashington%2C+GA+30673%2C+USA%3AUS%3AGeorgia%3AWilkes+County%3AWashington%3Anull%3A30673%3A33.7367948%3A-82.73930889999997%7CWashington%2C+VA+22747%2C+USA%3AUS%3AVirginia%3ARappahannock+County%3AWashington%3Anull%3A22747%3A38.7134519%3A-78.15944389999999%7CWashington%2C+WV+26181%2C+USA%3AUS%3AWest+Virginia%3AWood+County%3AWashington%3Anull%3A26181%3A39.244853%3A-81.66377649999998&change=yes"
    palo_alto_url = "https://www.meetup.com/find/events/?allMeetups=true&radius="+str(radius)+"&userFreeform=palo+alto&gcResults=Palo+Alto%2C+CA%2C+USA%3AUS%3ACalifornia%3ASanta+Clara+County%3APalo+Alto%3Anull%3Anull%3A37.4418834%3A-122.14301949999998%7CPalo+Alto%2C+PA%2C+USA%3AUS%3APennsylvania%3ASchuylkill+County%3APalo+Alto%3Anull%3Anull%3A40.6873122%3A-76.17216410000003&change=yes"
    boston_url = "https://www.meetup.com/find/events/?allMeetups=true&radius="+str(radius)+"&userFreeform=boston&gcResults=Boston%2C+MA%2C+USA%3AUS%3AMassachusetts%3ASuffolk+County%3ABoston%3Anull%3Anull%3A42.3600825%3A-71.05888010000001%7CBoston%2C+NY+14025%2C+USA%3AUS%3ANew+York%3AErie+County%3ABoston%3Anull%3A14025%3A42.6289495%3A-78.73752890000003%7CBoston%2C+KY+40107%2C+USA%3AUS%3AKentucky%3ANelson+County%3ABoston%3Anull%3A40107%3A37.787563%3A-85.67273740000002%7CBoston%2C+GA+31626%2C+USA%3AUS%3AGeorgia%3AThomas+County%3ABoston%3Anull%3A31626%3A30.7918613%3A-83.78988679999998%7CBoston%2C+VA+22713%2C+USA%3AUS%3AVirginia%3ACulpeper+County%3ABoston%3Anull%3A22713%3A38.5421671%3A-78.13227330000001%7CBoston%2C+IN%2C+USA%3AUS%3AIndiana%3AWayne+County%3ABoston%3Anull%3Anull%3A39.74116009999999%3A-84.85190339999997&change=yes"
    la_url = "https://www.meetup.com/find/events/?allMeetups=true&radius="+str(radius)+"&userFreeform=Los+Angeles%2C+California%2C+USA&mcId=c90001&gcResults=Boston%2C+MA%2C+USA%3AUS%3AMassachusetts%3ASuffolk+County%3ABoston%3Anull%3Anull%3A42.3600825%3A-71.05888010000001%7CBoston%2C+NY+14025%2C+USA%3AUS%3ANew+York%3AErie+County%3ABoston%3Anull%3A14025%3A42.6289495%3A-78.73752890000003%7CBoston%2C+KY+40107%2C+USA%3AUS%3AKentucky%3ANelson+County%3ABoston%3Anull%3A40107%3A37.787563%3A-85.67273740000002%7CBoston%2C+GA+31626%2C+USA%3AUS%3AGeorgia%3AThomas+County%3ABoston%3Anull%3A31626%3A30.7918613%3A-83.78988679999998%7CBoston%2C+VA+22713%2C+USA%3AUS%3AVirginia%3ACulpeper+County%3ABoston%3Anull%3A22713%3A38.5421671%3A-78.13227330000001%7CBoston%2C+IN%2C+USA%3AUS%3AIndiana%3AWayne+County%3ABoston%3Anull%3Anull%3A39.74116009999999%3A-84.85190339999997&change=yes"
    mv_url = "https://www.meetup.com/find/events/?allMeetups=true&radius="+str(radius)+"&userFreeform=mountain+view&gcResults=Mountain+View%2C+CA%2C+USA%3AUS%3ACalifornia%3ASanta+Clara+County%3AMountain+View%3Anull%3Anull%3A37.3860517%3A-122.0838511%7CMountain+View%2C+AR+72560%2C+USA%3AUS%3AArkansas%3AStone+County%3AMountain+View%3Anull%3A72560%3A35.8684075%3A-92.11765209999999%7CMountain+View%2C+HI%2C+USA%3AUS%3AHawaii%3AHawaii+County%3AMountain+View%3Anull%3Anull%3A19.5558333%3A-155.1080556%7CMountain+View%2C+WY+82933%2C+USA%3AUS%3AWyoming%3AUinta+County%3AMountain+View%3Anull%3A82933%3A41.2688392%3A-110.3398785%7CMountain+View%2C+CO+80212%2C+USA%3AUS%3AColorado%3AJefferson+County%3AMountain+View%3Anull%3A80212%3A39.7744311%3A-105.05553889999999%7CMountain+View%2C+OK+73062%2C+USA%3AUS%3AOklahoma%3AKiowa+County%3AMountain+View%3Anull%3A73062%3A35.0975577%3A-98.74645520000001&change=yes"
    sea_url = "https://www.meetup.com/find/events/?allMeetups=true&radius="+str(radius)+"&userFreeform=seattle&gcResults=Seattle%2C+WA%2C+USA%3AUS%3AWashington%3AKing+County%3ASeattle%3Anull%3Anull%3A47.6062095%3A-122.3320708&change=yes"
    austin_url = "https://www.meetup.com/find/events/?allMeetups=true&radius="+str(radius)+"&userFreeform=austin&gcResults=Austin%2C+TX%2C+USA%3AUS%3ATexas%3ATravis+County%3AAustin%3Anull%3Anull%3A30.267153%3A-97.74306079999997%7CAustin%2C+MN+55912%2C+USA%3AUS%3AMinnesota%3AMower+County%3AAustin%3Anull%3A55912%3A43.6666296%3A-92.97463670000002%7CAustin%2C+IN+47102%2C+USA%3AUS%3AIndiana%3AScott+County%3AAustin%3Anull%3A47102%3A38.7415038%3A-85.80595819999996%7CAustin%2C+NV+89310%2C+USA%3AUS%3ANevada%3ALander+County%3AAustin%3Anull%3A89310%3A39.49257009999999%3A-117.0672356%7CAustin%2C+AR+72007%2C+USA%3AUS%3AArkansas%3ALonoke+County%3AAustin%3Anull%3A72007%3A34.9984206%3A-91.98375520000002%7CAustin%2C+KY+42123%2C+USA%3AUS%3AKentucky%3ABarren+County%3AAustin%3Anull%3A42123%3A36.8253279%3A-86.01914779999998&change=yes"
    url_dict = {'ny': ny_url, 'sf': sf_url, 'chicago': chicago_url, 'dc': dc_url, 'palo_alto': palo_alto_url,\
              'boston': boston_url, 'la': la_url, 'mountain_view': mv_url, 'seattle': sea_url, 'austin': austin_url}
    return url_dict

In [3]:
def get_meet_up_soup(url, to_which_date):
    '''create the master meetup soup'''
    # create driver for website
    driver = webdriver.Chrome('/Users/annecool37/Documents/chromedriver')
    # wait 10 sec 
    driver.implicitly_wait(10)
    # pass the url to Chrome
    driver.get(url)
    # locate and click the "Show more" button at the bottom
    btn = driver.find_element_by_class_name('simple-post-result-wrap')
    btn.click()
    driver.implicitly_wait(10)
    html = driver.page_source
    temp_soup = BeautifulSoup(html, 'lxml')
    # Scrape all data till 'to_which_date'
    while to_which_date not in temp_soup.find('div', {'id':'docked-event-date'}).get_text():    
        # scroll to the bottom to reveal more events
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        # update html
        html = driver.page_source
        # convert html into a beautifulsoup object
        temp_soup = BeautifulSoup(html, 'lxml')
    return temp_soup

In [4]:
def catch(func, handle=lambda e : e, *args, **kwargs):
    '''Catch exception and return NA if no such attribute is found'''
    try:
        return func(*args, **kwargs)
    except Exception as e:
        return 'NA'

In [5]:
def catch_yes_no(func, handle=lambda e : e, *args, **kwargs):
    '''Catch exception. Return yes if the option exists, else return no'''
    try:
        func(*args, **kwargs)
        return 'Yes'
    except Exception as e:
        return 'No'

In [6]:
def get_name_title(soup):
    '''get group name and event title'''
    titleTags = soup.find_all('span', {'itemprop': "name"})
    event_titles = []
    group_names = []
    for i in range(0,len(titleTags),2):
        group_names.append(titleTags[i].get_text()) 
        event_titles.append(titleTags[i+1].get_text()) 
    return [group_names, event_titles]

In [7]:
def get_participant_count(meet_up_soup):
    '''get number of participants '''
    numPeopleTags = meet_up_soup.find_all('div', {'class': "attendee-count"})
    participant_count = [catch(lambda: int(re.search(r'\d+', tag.get_text()).group()))\
                         for tag in numPeopleTags]
    return participant_count

In [8]:
def event_url_to_html(meet_up_soup):
    '''convert all event url to beautifulsoup objects'''
    # get event urls
    event_soup = meet_up_soup.find_all('div', 'row-item row-item--shrink text--secondary')
    event_url_lst = [(e.a)['href'] for e in event_soup]
    # parse all event url the into html
    event_html_lst = [BeautifulSoup(requests.get(url).text, 'lxml') for url in event_url_lst]
    return event_html_lst

In [10]:
def get_event_time(event_html_lst): 
    '''get event date and event start time'''
    event_time_soup = [catch(lambda: html.find('time')) for html in event_html_lst]
    # date format example 2016-08-05
    event_date_lst = [catch(lambda: time['datetime'][0:10]) for time in event_time_soup]
    # start time format example 15:30
    start_time_lst = [catch(lambda: time['datetime'][11:16]) for time in event_time_soup]
    return [event_date_lst, start_time_lst]

In [11]:
def get_address(event_html_lst):
    '''get event address and extract the county and state'''
    location_soup = [html.find_all('p', {'class':'event-where-address small'})\
                     for html in event_html_lst]
    location_lst = [catch(lambda: location[0].get_text()) for location in location_soup]
    county_state_lst = [catch(lambda: location.split(",")[-2][1:]+','+location.split(",")[-1][0:3]) for location in location_lst]
    return county_state_lst

In [12]:
def get_price(event_html_lst):
    '''get price of event, record as NA if no price is listed'''
    price_soup = [html.find('div', {'id':'event-fee-display'}) for html in event_html_lst]
    price_lst = [catch(lambda: float(str(price_tag.get_text()).split()[1][1:])) for price_tag in price_soup]
    return price_lst

In [13]:
def get_comment_reply_count(event_html_lst):
    '''get number of comments and replies in events'''
    comment_soup = [html.find_all('ul', {'id':'conversation'}) for html in event_html_lst]
    # get count of comment and reply
    comment_count_lst =[catch(lambda: len(comment_tags[0].find_all('li', {'data-commenttype': 'COMMENT'})))\
                        for comment_tags in comment_soup]
    reply_count_lst =[catch(lambda: len(comment_tags[0].find_all('li', {'data-commenttype': 'REPLY'})))\
                      for comment_tags in comment_soup]
    return [comment_count_lst, reply_count_lst]

In [14]:
def get_group_info(event_html_lst):
    '''get basic info of the group hosting this event
    get meet_up group size, num of group review,
    num of up coming meetups, and num of past meetups
    '''
    member_soup = [html.find_all('ul', {'class':'paddedList small margin-bottom'}) for html in event_html_lst]
    gr_size_lst = [catch(lambda: soup[0].find_all('span')[1].get_text()) for soup in member_soup]
    gr_review_count_lst = [catch(lambda: soup[0].find_all('span')[3].get_text()) for soup in member_soup]
    gr_upcoming_count_lst = [catch(lambda: soup[0].find_all('span')[5].get_text()) for soup in member_soup]
    gr_past_meetup_count_lst = [catch(lambda: soup[0].find_all('span')[7].get_text()) for soup in member_soup]
    return [gr_size_lst, gr_review_count_lst, gr_upcoming_count_lst, gr_past_meetup_count_lst]

In [15]:
def get_founded_time(event_html_lst):
    '''get the founded date for the event-hosting group'''
    # Note: if the code is too slow, remove all catch function since all group should have the founded year...
    founded_date_tags = [catch(lambda: html.find_all('div', {'class':'small margin-bottom'})[0]) for html in event_html_lst]
    date_founded = [catch(lambda: reduce(lambda x,y: x+" "+y, tag.get_text().split()[1:])) for tag in founded_date_tags]
    date_founded_formatted_lst = [catch(lambda: datetime.datetime.strptime(date, '%b %d, %Y').strftime('%Y-%m-%d')) for date in date_founded] 
    return date_founded_formatted_lst

In [16]:
def get_keywords(event_html_lst):
    '''get group keywords'''
    keyword_soup = [html.find_all('div', {'id':re.compile('^topicList')})\
                    for html in event_html_lst]
    keyword_tags = [catch(lambda: soup[0].find_all('a')) for soup in keyword_soup]
    # a nested lst: [[keyword for group 1], [keyword for group 2],.....]
    group_keywords = [[catch(lambda: tag.get_text()) for tag in keyword]\
                      for keyword in keyword_tags]
    return group_keywords

In [17]:
def get_sponsor_count(event_html_lst):
    '''get number of sponsors in the group that hosts this event'''
    sponsor_urls = [catch(lambda: html.find_all('a', {'href': re.compile('sponsors')})[0].get('href'))\
                          for html in event_html_lst]
    # convert teh sponsor link into beautiful soup object
    sponsor_htmls = [catch(lambda: BeautifulSoup(requests.get(url).text, 'lxml')) for url in sponsor_urls]
    # count the number of sponsor
    sponsor_count_lst = [catch(lambda: len(html.find('ul', {'id': 'sponsors-list'}).find_all('li'))) \
                         for html in sponsor_htmls]
    return sponsor_count_lst
#     # a simple list recording if the group is being sponsored (yes/no)
#     sponsor_yn_lst = [catch_yes_no(lambda: len(html.find('ul', {'id': 'sponsors-list'}).find_all('li'))) for html in sponsor_htmls]    
#     return [sponsor_count_lst, sponsor_yn_lst]

In [18]:
def check_policy_privacy(event_html_lst):
    '''check group approval policy and event privacy'''
    # private event is depedent on approval factor...?
    is_private_group = [catch_yes_no(lambda: html.find('p',{'class':'urf-notice urf-notice-private'}).get_text())\
                        for html in event_html_lst]
    approval_needed_group = [catch_yes_no(lambda: html.find('p',{'class':'urf-notice urf-notice-approval-only'}).get_text())\
                             for html in event_html_lst]
    # open_group = [catch_yes_no(lambda: html.find('p',{'class':'urf-notice urf-notice-open'}).get_text()) for html in event_html_lst]
    # open group did not have the opposite result of approval needed group... why?
    # only the private evnt will have notoce-approval & notice-open options?
    return [is_private_group, approval_needed_group]

In [19]:
def get_related_group(event_html_lst):
    '''get related group: check which meetup groups that people in this group are also in'''
    other_soup = [catch(lambda: html.find_all('div', {'class':'nearby-description'})) for html in event_html_lst]
    other_meetup_lst = [[catch(lambda: tag.find('a', {'class':'omnCamp omngj_sj35'}).get_text()) for tag in tags] for tags in other_soup]
    return other_meetup_lst

In [20]:
def to_dict(key_lst, value_lst):
    '''create dictionary to flatten nested list'''
    dic = {}
    for i in range(0,len(key_lst)):
        dic[key_lst[i]] = value_lst[i]
    return dic

In [21]:
def write_json(city, which, dic):
    '''write dictionary to json file'''
    filename = city + which + '.json'
    with open(filename, 'w') as f:
        json.dump(dic, f)

In [22]:
def unicode_to_ascii(lst):
    '''convert unicode to ascii'''
    # avoid raising errors later on while writing data into csv files
    return [item.encode('ascii', 'ignore') for item in lst]

In [23]:
def main_function(radius, city, to_which_date):
    '''The main function to get, store, and save data from meetup.com'''
    # get start time
    start_time = time.time()

    print "creating url dictionary..."
    url_dict = create_url_dict(radius)

    print "creating meet up soup..."
    meet_up_soup = get_meet_up_soup(url_dict[city], to_which_date)

    print "getting group name, event title, and number of participants"
    [group_names, event_titles]= get_name_title(meet_up_soup)
    participant_counts = get_participant_count(meet_up_soup)

    print "converting url to html..."
    event_html_lst = event_url_to_html(meet_up_soup)

    print "getting event date..."
    [event_date_lst, event_start_time_lst] = get_event_time(event_html_lst)

    print "getting event county & state..."
    county_state_lst = get_address(event_html_lst)

    print "getting event price"
    price_lst = get_price(event_html_lst)

    print "getting event comments..."
    [comment_count_lst, reply_count_lst]= get_comment_reply_count(event_html_lst)

    print "getting basic info of the group hosting the event..."
    [member_count_lst, review_count_lst, upcoming_event_count_lst, past_meetup_count_lst] = get_group_info(event_html_lst)

    print "getting group founded date..."
    group_founded_date_lst = get_founded_time(event_html_lst)

    print "getting group keywords..."
    group_keywords_lst = get_keywords(event_html_lst)

    print "checking sponsorships...."
    sponsor_count_lst = get_sponsor_count(event_html_lst)

    print "checking group and event privacy..."
    [is_private_group, approval_needed_group]= check_policy_privacy(event_html_lst)

    print "getting related meetup groups..."
    related_meetup_lst = get_related_group(event_html_lst)
    
    # create a dictionary
    data = {'group_name': group_names , 'event_title': event_titles,
            'participant_count': participant_counts, 'event_date': event_date_lst, 'event_start_time': event_start_time_lst, 
            'location': county_state_lst, 'price': price_lst, 'comment_count': comment_count_lst, 
            'reply_count': reply_count_lst, 'member_count': member_count_lst, 'review_count': review_count_lst, 
            'upcoming_event_count': upcoming_event_count_lst, 'past_meetup_count': past_meetup_count_lst, 
            'founded_date': group_founded_date_lst, 'group_keywords': group_keywords_lst, 
            'sponsor_count': sponsor_count_lst, 'private_group': is_private_group, 
            'approval_needed': approval_needed_group, 'related_meetup': related_meetup_lst}
    
    # convert unicode to ascii if needed
    data['group_name'] = unicode_to_ascii(data['group_name'])
    data['event_title'] = unicode_to_ascii(data['event_title'])
    data['location'] = unicode_to_ascii(data['location'])
    data['group_keywords'] = [unicode_to_ascii(keywords) for keywords in data['group_keywords']]    
    data['related_meetup'] = [unicode_to_ascii(group) for group in data['related_meetup']]    
    
    # {meetup group: other meetup groups that people in this group has joined}
    group_dict = to_dict(data['group_name'] , data['related_meetup'])
    # {meetup group: keywords of this group}
    keyword_dict = to_dict(data['group_name'] , data['group_keywords'])
    # write to json files
    write_json(city, 'group', group_dict)
    write_json(city, 'keyword', keyword_dict)
    
    # convert dictionary to DataFrame
    df = pd.DataFrame(data)
    print "saving data to csv..."
    # save file to .csv
    df.to_csv(city+'_meetup.csv')
    print "done!"
    # record end time
    end_time = time.time()
    print "time spent: " + str((end_time - start_time))
    return data   

In [25]:
# Get data from meetup.com
# define criteria
radius = 10
to_which_month = 'October' 
# actually retrieve data till late October or even later due to the fast page scrolling speed
url_dict = create_url_dict(radius)
result_dict = {}
for key in url_dict.keys():
    # get data on meetup.com and save it as dictionary
    data = main_function(radius, key, to_which_month)
    # store result data
    result_dict[key] = data
    print "collected " + str(len(data['group_name'])) + " data for " + key

creating url dictionary...
creating meet up soup...
getting group name, event title, and number of participants
converting url to html...
getting event date...
getting event county & state...
getting event price
getting event comments...
getting basic info of the group hosting the event...
getting group founded date...
getting group keywords...
checking sponsorships....
checking group and event privacy...
getting related meetup groups...
saving data to csv...
done!
time spent: 972.634635925
collected 951 data for palo_alto
creating url dictionary...
creating meet up soup...
getting group name, event title, and number of participants
converting url to html...
getting event date...
getting event county & state...
getting event price
getting event comments...
getting basic info of the group hosting the event...
getting group founded date...
getting group keywords...
checking sponsorships....
checking group and event privacy...
getting related meetup groups...
saving data to csv...
done!
t

In [68]:
url_dict.keys()

['palo_alto',
 'ny',
 'seattle',
 'chicago',
 'boston',
 'mountain_view',
 'la',
 'austin',
 'sf',
 'dc']

In [26]:
# check the result data
# data = result_dict['palo_alto']
df = pd.DataFrame(data)
print df.head(n=5)
# double check how many was collected
print df.shape

  approval_needed comment_count  event_date event_start_time  \
0              No             0  2016-08-10            21:00   
1              No             3  2016-08-10            21:00   
2              No             0  2016-08-10            21:00   
3              No             0  2016-08-10            21:00   
4              No             0  2016-08-11            06:15   

                                         event_title founded_date  \
0                Karaoke at Flanagan's Harp & Fiddle   2012-01-15   
1                                     Diet Cig @ DC9   2008-02-24   
2  Wednesday Night Swing Dancing at Nick's Nightc...   2006-06-20   
3           Yoga for Beginners (Free Online Webinar)   2016-03-17   
4        14th Street Morning Track Workout- Thursday   2006-04-06   

                                      group_keywords  \
0  [Sci-Fi/Fantasy, Social Networking, New In Tow...   
1  [Indie Music, Music, Live Music, Concerts, Alt...   
2                               

In [27]:
##################################
##################################
## test code for just one event ##
##################################
##################################
# create example html for testing
# test_url = "https://www.meetup.com/New-York-City-Hiking/events/230743402/"
test_url = "http://www.meetup.com/New-York-Hikers-and-Backpackers-Club/"
test_html = BeautifulSoup(requests.get(test_url).text, 'lxml') 
# get address 
location = test_html.find_all('p', {'class':'event-where-address small'})
address = [place.get_text() for place in location]
print address

# get group keywords 
keyword_soup = test_html.find_all('div', {'id':re.compile('^topicList')})
keywords_tags = [word.find_all('a') for word in keyword_soup]
keywords = [tag.get_text() for tag in keywords_tags[0]]
print keywords

# get event price
price_tag = test_html.find('div', {'id':'event-fee-display'})
# ['Price:', '$17.00', '/per', 'person', 'Refund', 'policy']
print catch(lambda: float(str(price_tag.get_text())).split()[1][1:])

# get meet up group size for one event
group_info = test_html.find_all('ul', {'class':'paddedList small margin-bottom'})
group_size = group_info[0].find_all('span')[1].get_text()
group_review = group_info[0].find_all('span')[3].get_text()
up_coming_meetup = group_info[0].find_all('span')[5].get_text()
past_meetups = group_info[0].find_all('span')[7].get_text()
print [group_size, group_review, up_coming_meetup, past_meetups]

# get comment and reply count
comment_soup = test_html.find_all('ul', {'id':'conversation'})
num_comment = len(catch(lambda: comment_soup[0].find_all('li', {'data-commenttype': 'COMMENT'})))
num_reply = len(catch(lambda: comment_soup[0].find_all('li', {'data-commenttype': 'REPLY'})))
print [num_comment, num_reply]

# get number of sponsor for one event
toy_url = "https://www.meetup.com/keepingupwithphilly/events/233042565/"
toy_html = BeautifulSoup(requests.get(toy_url).text, 'lxml') 
sponsor_url = toy_html.find_all('a', {'href': re.compile('sponsors')})[0].get('href')
sponsor_html = BeautifulSoup(requests.get(sponsor_url).text, 'lxml')
sponsor_count = len(sponsor_html.find('ul', {'id': 'sponsors-list'}).find_all('li'))
print sponsor_count

# check event privacy
private_url = "https://www.meetup.com/AMC-Sunday-Softball-in-Central-Park/"
private_html = BeautifulSoup(requests.get(private_url).text, 'lxml') 

# Content of this Meetup Group is only visible to members
print private_html.find('p',{'class':'urf-notice urf-notice-approval-only'}).get_text()
# New members must be approved by the Organizer
print private_html.find('p',{'class':'urf-notice urf-notice-private'}).get_text()

# get the time that the group was founded for this event
toy_url = "https://www.meetup.com/keepingupwithphilly/events/233042565/"
toy_html = BeautifulSoup(requests.get(toy_url).text, 'lxml') 
date_tags = toy_html.find_all('div', {'class':'small margin-bottom'})[0]
date_founded = reduce(lambda x,y: x+" "+y, date_tags.get_text().split()[1:])
date_founded_formatted = datetime.datetime.strptime(date_founded, '%b %d, %Y').strftime('%Y-%m-%d')
print date_founded_formatted

# People in this Meetup are also in which meetup? #
toy_url = "https://www.meetup.com/CityKinder-New-York-Meetup-Group/events/232727728/"
toy_html = BeautifulSoup(requests.get(toy_url).text, 'lxml') 
soup = toy_html.find_all('div', {'class':'nearby-description'})
other_groups = [tag.find('a', {'class':'omnCamp omngj_sj35'}).get_text() for tag in soup]
print other_groups

[]
[u'Camping', u'Kayaking', u'Hiking', u'Rock Climbing', u'Adventure', u'Weekend Adventures', u'Outdoor Adventures', u'Water Sports', u'Backpacking', u'Day hikes', u'Mountain Biking', u'Road Trips', u'Winter Sports', u'Skiing Snowboarding SnowShoeing & CrossCountry', u'WILDERNESS HIKING']
NA
[u'1,511', u'4', u'15', u'131']
[2, 2]
3
New members must be approved by the Organizer
Content of this Meetup Group is only visible to members
2011-08-26
[u'New York City German Culture Group', u'Manhattan Mamas', u'New York City Culture & Social Club (6,000+ members)', u'Free Events for Mom', u'German speaking playgroup - Jersey City/Hoboken', u'Yoga NYC Meetup']


In [28]:
# # create driver for website
# driver = webdriver.Chrome('/Users/annecool37/Documents/chromedriver')
# # wait 10 sec 
# driver.implicitly_wait(10)
# # pass the url to Chrome
# driver.get(ny_url)
# # locate and click the "Show more" button at the bottom
# btn = driver.find_element_by_class_name('simple-post-result-wrap')
# btn.click()
# driver.implicitly_wait(10)
# ## run particular round to scroll the page      
# #for i in range(10):
# #    # scroll to the bottom to reveal more events
# #    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
# #    # wait till page is load
# #    time.sleep(5)
# #    # WebDriverWait(driver, 30)
# #    print i
# html = driver.page_source
# temp_soup = BeautifulSoup(html, 'lxml')
# # Scrape all data before September from August Xth 
# # while "September" not in temp_soup.find('div', {'id':'docked-event-date'}).get_text():
# while "August 7" not in temp_soup.find('div', {'id':'docked-event-date'}).get_text():    
#     # scroll to the bottom to reveal more events
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#     # update html
#     html = driver.page_source
#     # convert html into a beautifulsoup object
#     temp_soup = BeautifulSoup(html, 'lxml')
#     # print temp_soup.find('div', {'id':'docked-event-date'}).get_text()

In [29]:
# # if were to extract the whole meetip.com
# # create driver for website
# driver = webdriver.Chrome('/Users/annecool37/Documents/chromedriver')
# # wait 10 sec 
# driver.implicitly_wait(10)
# driver.get("https://www.meetup.com/")
# # locate and click the calendar
# elem = driver.find_element_by_id('simple-view-selector-event') 
# elem.click()
# # locate and click show more
# elem = driver.find_element_by_class_name('simple-post-result-wrap') 
# elem.click()

In [30]:
# # this is a short list of event just for setting up the code
# meet_up_soup = requests.get(ny_url).text
# meet_up_soup = BeautifulSoup(meet_up_soup,'lxml')

In [9]:
# this function works, however, the event time collected is in EST
# didnot convert accordingly to where city located....
# def get_event_time(meet_up_soup): 
#     '''get event date and time'''
#     event_soup = meet_up_soup.find_all('div', 'row-item row-item--shrink text--secondary')
#     event_time_lst = [(e.time)['datetime'] for e in event_soup]
#     # date format example 2016-08-05
#     event_date_lst = [time[0:10] for time in event_time_lst]
#     # start time format example 15:30
#     event_start_time_lst = [time[11:16] for time in event_time_lst]
#     # convert to weekday (Monday - Sunday)
#     event_day_lst = [datetime.datetime.strptime(date, '%Y-%m-%d').strftime('%A')\
#                      for date in event_date_lst]    
#     return [event_date_lst, event_start_time_lst, event_day_lst]
# # datetime syntax reference : http://strftime.org

In [31]:
##################################
## get group name (another way) ##
##################################
# meet_up_soup = temp_soup
# group_names = []
# for i, tag in enumerate(meet_up_soup.findAll('li', { 'class': 'event-listing' })):
#     group_names.append(tag.find('span', {'itemprop': 'name'}).string)
# num_group_name = len(group_name)
# print 'number of observations scrapped: ' + str(num_group_name)
# print  num_group_name == event_count
# group_names[0:10]

In [32]:
######################
# Miscellaneous Note #
######################
# find highest freqency (top 10?) word in event name
# find highest frequency (top 10?) word in location
# find the lontitude and latitude
# check unique time
# bar plot: number of participant vs. weekday
# bar plot: number of participant vs. duration
# other thoughts
# implement weather into analysis?
# model building
# subset data into training (August ones?) and testing set (September ones?)
# and...?

In [24]:
# def main_function(radius, city, to_which_date):
#     '''The main function to get, store, and save data from meetup.com'''
#     # get start time
#     start_time = time.time()

#     print "creating url dictionary..."
#     url_dict = create_url_dict(radius)

#     print "creating meet up soup..."
#     meet_up_soup = get_meet_up_soup(url_dict[city], to_which_date)

#     print "getting group name, event title, and number of participants"
#     [group_names, event_titles]= get_name_title(meet_up_soup)
#     participant_counts = get_participant_count(meet_up_soup)

#     print "converting url to html..."
#     event_html_lst = event_url_to_html(meet_up_soup)

#     print "getting event date..."
#     [event_date_lst, event_start_time_lst, event_day_lst] = get_event_time(meet_up_soup)

#     print "getting event county & state..."
#     county_state_lst = get_address(event_html_lst)

#     print "getting event price"
#     price_lst = get_price(event_html_lst)

#     print "getting event comments..."
#     [comment_count_lst, reply_count_lst]= get_comment_reply_count(event_html_lst)

#     print "getting basic info of the group hosting the event..."
#     [member_count_lst, review_count_lst, upcoming_event_count_lst, past_meetup_count_lst] = get_group_info(event_html_lst)

#     print "getting group founded date..."
#     group_founded_date_lst = get_founded_time(event_html_lst)

#     print "getting group keywords..."
#     group_keywords_lst = get_keywords(event_html_lst)

#     print "checking sponsorships...."
#     sponsor_count_lst = get_sponsor_count(event_html_lst)

#     print "checking group and event privacy..."
#     [is_private_group, approval_needed_group]= check_policy_privacy(event_html_lst)

#     print "getting related meetup groups..."
#     related_meetup_lst = get_related_group(event_html_lst)
    
#     # create a dictionary
#     data = {'group_name': group_names , 'event_title': event_titles,
#             'participant_count': participant_counts, 'event_date': event_date_lst, 'event_start_time': event_start_time_lst, 
#             'event_day':event_day_lst , 'location': county_state_lst, 'price': price_lst, 'comment_count': comment_count_lst, 
#             'reply_count': reply_count_lst, 'member_count': member_count_lst, 'review_count': review_count_lst, 
#             'upcoming_event_count': upcoming_event_count_lst, 'past_meetup_count': past_meetup_count_lst, 
#             'founded_date': group_founded_date_lst, 'group_keywords': group_keywords_lst, 
#             'sponsor_count': sponsor_count_lst, 'private_group': is_private_group, 
#             'approval_needed': approval_needed_group, 'related_meetup': related_meetup_lst}
    
#     # convert unicode to ascii if needed
#     data['group_name'] = unicode_to_ascii(data['group_name'])
#     data['event_title'] = unicode_to_ascii(data['event_title'])
#     data['location'] = unicode_to_ascii(data['location'])
#     data['group_keywords'] = [unicode_to_ascii(keywords) for keywords in data['group_keywords']]    
#     data['related_meetup'] = [unicode_to_ascii(group) for group in data['related_meetup']]    
    
#     # {meetup group: other meetup groups that people in this group has joined}
#     group_dict = to_dict(data['group_name'] , data['related_meetup'])
#     # {meetup group: keywords of this group}
#     keyword_dict = to_dict(data['group_name'] , data['group_keywords'])
#     # write to json files
#     write_json(city, 'group', group_dict)
#     write_json(city, 'keyword', keyword_dict)
    
#     # convert dictionary to DataFrame
#     df = pd.DataFrame(data)
#     print "saving data to csv..."
#     # save file to .csv
#     df.to_csv(city+'_meetup.csv')
#     print "done!"
#     # record end time
#     end_time = time.time()
#     print "time spent: " + str((end_time - start_time))
#     return data   